In [ ]:
%cd /home/jovyan/onceuponai/websites/src/stories-fantasy-shop-assistant/notebooks
from IPython.core.display import HTML
HTML("""
<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Bangers" />
<style>
h1 {font-family: Bangers !important; font-size: 150px !important}
h2 {font-family: Bangers !important; font-size: 120px !important}
h3 {font-family: Bangers !important; font-size: 100px !important}
h4 {font-family: Bangers !important; font-size: 80px !important}
h5 {font-family: Bangers !important; font-size: 50px !important}
h6 {font-family: Bangers !important; font-size: 30px !important}
.txt7 {font-family: Bangers !important; font-size: 70px !important}
.txt6 {font-family: Bangers !important; font-size: 60px !important}
.txt5 {font-family: Bangers !important; font-size: 50px !important}
.txt4 {font-family: Bangers !important; font-size: 40px !important}
.txt3 {font-family: Bangers !important; font-size: 30px !important}
.cap {font-family: Bangers !important; font-size: 30px !important}
.controls
{
    display: none !important;
}

.slide-number
{
    display: none !important;
}
.inline-block {
   display: inline-block;
}

.backgrounds {
   background-color: #fff !important;
}

.controls {
  display: none !important;
}
</style>
""")

import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install deltalake polars

# LLMs

In [ ]:
import onceuponai

bielik = onceuponai.llms.Quantized("speakleash/Bielik-7B-Instruct-v0.1-GGUF", \
                              "bielik-7b-instruct-v0.1.Q4_K_S.gguf", \
                              "speakleash/Bielik-7B-Instruct-v0.1", "cuda")

In [6]:
import onceuponai

bielik = onceuponai.llms.Quantized("speakleash/Bielik-7B-Instruct-v0.1-GGUF", \
                              "file:///home/jovyan/rust-src/Bielik/Bielik-7B-Instruct-v0.1.Q4_K_S.gguf", \
                              "speakleash/Bielik-7B-Instruct-v0.1", "cuda")

In [ ]:
res = await bielik.invoke("Jak ugotować żurek ?", 1000)
res

In [ ]:
import onceuponai

krakowiak = onceuponai.llms.Quantized("szymonrucinski/Krakowiak-7B-v2", \
                              "file:///home/jovyan/rust-src/Bielik/Krakowiak-7B-v2.Q4_K_S.gguf", \
                              "szymonrucinski/Krakowiak-7B-v2", "cuda")

In [ ]:
res = await krakowiak.invoke("Jak ugotować żurek ?", 1000)
res

In [ ]:
import onceuponai

curie = onceuponai.llms.Quantized("szymonrucinski/Curie-7B-v1", \
                              "file:///home/jovyan/rust-src/Bielik/Curie-7B-v1.Q3_K.gguf", \
                              "szymonrucinski/Curie-7B-v1", "cuda")

In [ ]:
res = await curie.invoke("Jak ugotować rosół ?", 1000)
res

In [ ]:
import onceuponai

await onceuponai.sleep(2)

## Prepare data

In [ ]:
import yaml
from deltalake.writer import write_deltalake
import pandas as pd
import polars as pl

data = yaml.safe_load(open("recipes.yaml", "r"))

prep={}


df = pl.DataFrame(data["Recipes"])
df.to_pandas()

# Embeddings

In [2]:
import onceuponai

a = onceuponai.embeddings.E5("intfloat/multilingual-e5-small")

In [ ]:
a.embed(["witaj przygodo"])

In [ ]:
items = []
for combined_text in df['Recipe']:
    emb = a.embed([combined_text])
    items.append({"item": combined_text, "vector": emb[0]})

items

# Fake words

In [ ]:
import random

subjects = [
    "The mischievous squirrel", "The colorful birds", "The playful puppies", 
    "The dedicated teacher", "The curious boy", "The brilliant inventor", "Girl", "Cat", "Mouse", "Dog"
]

verbs = [
    "danced", "explored", "built", "discovered", "imagined", 
    "sang", "cooked", "programmed","run", "sing", "walk", 
]

objects = ["the sandwich", "a brilliant app", "over the fence", "a beautiful song", "in the sky"]

for i in range(300):
    phrase = random.choice(subjects) + " " + random.choice(verbs) + " " + random.choice(objects)
    emb = a.embed([phrase])
    items.append({"item": phrase, "vector": emb[0]})

# Lance DB

In [ ]:
import os
import getpass

print("ACCOUNT NAME")
os.environ["AZURE_STORAGE_ACCOUNT_NAME"]=getpass.getpass()

print("ACCOUNT KEY")
os.environ["AZURE_STORAGE_ACCOUNT_KEY"]=getpass.getpass()

In [ ]:
import lancedb

db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.create_table("recipes_vectors", data=items)

In [ ]:
res = await tbl.create_index("vector")

# Search

In [ ]:
import lancedb

emb = a.embed(["Chciałbym zrobić żurek"])
db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.open_table("recipes_vectors")
res = await tbl.query().nearest_to(emb[0]).limit(1).to_pandas()
res

In [ ]:
import lancedb

db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.open_table("recipes_vectors")
template = """[INST]Skorzystaj z poniższych fragmentów kontekstu, aby odpowiedzieć na pytanie na końcu. 
Jeśli nie znasz odpowiedzi, po prostu powiedz, że nie wiesz, nie próbuj wymyślać odpowiedzi. 

Kontekst: 
{context}

Pytanie:
{question}[/INST]"""

async def chat(lancedb_table, prompt_template, embedding_model, llm_model, prompt):
    emb = embedding_model.embed([prompt])
    res = await lancedb_table.query().nearest_to(emb[0]).limit(1).to_pandas()
    context = res.to_dict()["item"][0]
    prompt = prompt_template.replace("{context}", context).replace("{question}", prompt)
    return await llm_model.invoke(prompt, 1000)
    

In [ ]:
await chat(tbl, template, a, bielik, "Jak ugotować żurek ?")

In [ ]:
await chat(tbl, template, a, bielik, "W jakiej temperaturze należy piec rybę ?")

In [ ]:
await chat(tbl, template, a, bielik, "Ile czasu trzeba piec kaczkę ?")

In [ ]:
import requests
import json

english_text = json.dumps('{"text": "Jak ugotować żurek ?"}')

response = requests.post("http://127.0.0.1:9090/", json=english_text)
french_text = response.text

print(french_text)